# Sesión 7.3 - BERT Finetuning para clasificación de textos

**En esta sesión vamos a reentrenar un modelo preentrenado de BERT/RoBERTa para poder obtener un clasificador de texto.**

Obsérvese que este enfoque es diferente al de la sesión anterior, en la que usábamos el modelo preentrenado como extractor de características para entrenar a un clasificador diferente (un SVM).

**Es importante tener en cuenta que para hacer el finetuning de modelos Transformers como BERT o RoBERTa es muy recomendable tener una GPU para reducir de manera drástica el tiempo de procesamiento. Por lo tanto se recomienda cargar este notebook en COLAB**

1. Instalamos las librerías transformers y datasets.
2. Descargamos el modelo MarIA basado en RobERTa y lo configuramos para dos etiquetas 'positivo' y 'negativo'.
3. Probamos el modelo sin ajustar con algunos textos.
4. Realizamos un finetuning del modelo con PyTorch, para ello:

  *   Preparamos conjuntos de entrenamiento, validación (devset) y prueba (instancias de subclase de PLNEDataset de PyTorch).
  *   Definimos los parámetros de entrenamiento, training_args.
  *   Creamos un Trainer e invocamos su método train

5. Salvamos el modelo ya ajustado.
6. Obtenemos las predicciones del modelo sobre el conjunto de prueba y a partir de ellas la precisión del modelo según la métrica que hayamos definido.
7. Utilizamos el modelo para hacer inferencias.
8. Creamos un modelo con TensorFlow a partir del modelo ya ajustado con PyTorch (se pueden cargar modelos de PyTorch en TensorFlow y viceversa).
9. Ajustamos otro modelo más pequeño (CenIA) y lo probamos. Básicamente repetimos pasos 4 a 7 con otro modelo.


In [ ]:
# Install libraries
!pip3 install transformers datasets torch
!pip3 install accelerate evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Modelo BERT en español - BETO
#path_bert_model = 'dccuchile/bert-base-spanish-wwm-uncased'

# Modelo BERT multilingüe
#path_bert_model = 'bert-base-multilingual-cased'

# Modelo BERTIN basado en RoBERTa
#path_bert_model = 'bertin-project/bertin-roberta-base-spanish'

# Modelo MarIA basado en RoBERTa
path_bert_model = 'PlanTL-GOB-ES/roberta-base-bne'

# Modelo "destilados" de BERT en español
#path_bert_model = 'CenIA/distillbert-base-spanish-uncased'

# Modelo AlBERT en español
#path_bert_model = 'CenIA/albert-base-spanish'

## Apartado 1.1 Cargarmos un modelo de BERT/RoBERTa preentrenado para clasificación binaria

A continuación vamos a cargar un modelo BERT/RoBERTa para clasificación binaria.

El finetuning y el uso de BERT/RoBERTa se puede hacer tanto con Pytorch como con KERAS. Empezaremos con pytorch y al final usaremos TensorFlow con el mismo modelo.



In [ ]:
# Cargamos el modelo para clasificación y lo configuramos para dos etiquetas
# 'positivo' y 'negativo'
import transformers
from transformers import (AutoModelForSequenceClassification,    # pytorch
                          TFAutoModelForSequenceClassification,  # TensorFlow
                          AutoTokenizer)

# Configuración de las labels e ids
NUM_LABELS = 2
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

# Cargamos y configuramos el modelo
modelo_preentrenado = AutoModelForSequenceClassification. \
                        from_pretrained(path_bert_model, num_labels=NUM_LABELS)
modelo_preentrenado.config.id2label = id2label
modelo_preentrenado.config.label2id = label2id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Probamos a clasificar unas frases, pero como el modelo preentrenado no se ha adaptado para hacer la tarea de clasificación, los resultados son muy malos.

In [ ]:
# Vamos a clasificar unas frases con el modelo preentrenado. El resultado será
# malo: hará falta fine-tuning
import torch

textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# Cargamos el Tokenizer asociado al modelo que estamos usando.
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

# Debug: para mostrar al final
inp = []
log = []
pred = []

for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  inp.append(inputs)

  with torch.no_grad():
    logits = modelo_preentrenado(**inputs).logits
    log.append(logits)

  predicted_class_id = logits.argmax().item()
  prediction= modelo_preentrenado.config.id2label[predicted_class_id]
  pred.append(prediction)

# Debug
print()
print("-------------------------------------------------------")
for i in range(len(textos)):
  print(inp[i])
  print(textos[i],' ----> ', pred[i], "\n", log[i].softmax(1))
  print("-------------------------------------------------------")


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]


-------------------------------------------------------
{'input_ids': tensor([[    0,  9252,  1581,   467,  6728,   383,   670, 30270,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
hay muchos más muertos por covid  ---->  negative 
 tensor([[0.5066, 0.4934]])
-------------------------------------------------------
{'input_ids': tensor([[    0,   434,  1656,   313,  6500,   383,   670, 30270,  8962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
el número de afectados por covid aumenta  ---->  negative 
 tensor([[0.5070, 0.4930]])
-------------------------------------------------------
{'input_ids': tensor([[    0, 26581,   320,  3484,   313,   332, 23079,  5527,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
vamos a salir de la pandemia  ---->  negative 
 tensor([[0.5114, 0.4886]])
-------------------------------------------------------
{'input_ids': tensor([[    0, 46541,   320,   784,     2]]), 'attention_mask': tensor(

## Apartado 1.2 Realizamos un Finetuning del modelo preentrenado de BERT/RoBERTa en Pytorch

Se puede observar por los resultados anteriores que el modelo de BERT/RoBERTA utilizado es general y no está adaptado para hacer clasificación de texto en 'positivo' y 'negativo' y mucho menos para el dominio del estado de alarma.

Por ello, vamos a realizar un finetuning de este modelo que mejorará muy significativamente sus resultados.

Realizaremos el finetuning con los datasets de tweets de prácticas anteriores. Para el entrenamiento de los modelos de DeepLearning es necesario tener 3 conjuntos de datos: Train, Eval y Test.

### 1.2.1 Descargamos los datasets para el entrenamiento.

In [ ]:
# Desde repositorio UMU
# Descargamos nuestro datasetEspañol.csv
!wget -c --no-check-certificate http://valencia.inf.um.es/valencia-plne/dataset_train.csv
!wget -c --no-check-certificate http://valencia.inf.um.es/valencia-plne/dataset_test.csv

In [ ]:
# En caso de que se haya descargado los ficheros en un directorio local
# asociado a drive.
from google.colab import drive
g_drive_path = "/content/drive"
drive.mount(g_drive_path)

# Directorio Drive Domus
!ls -l "/content/drive/Othercomputers/your_path"
data_dir_path = "/content/drive/Othercomputers/your_path"

Mounted at /content/drive
total 23
drwx------ 2 root root 4096 Mar 24 15:06 p1_5
drwx------ 2 root root 4096 Mar 25 07:32 p1_6
drwx------ 2 root root 4096 Mar 24 10:33 p1_7
drwx------ 2 root root 4096 Feb 26 08:13 P2
-rw------- 1 root root 2294 Mar  5 11:33 spain_reddit.json
drwx------ 2 root root 4096 Feb 26 08:13 T3


### 1.2.2 Preparamos los conjuntos de entrenamiento, evaluación y test en **Pytorch** para poder hacer el ajuste del modelo.

csv --> dataframe --> barajamos datos --> añadimos codificación numérica de las etiquetas --> dividimos trainset en trainset y devset.

In [ ]:
import pandas
df_train = pandas.read_csv(data_dir_path + "dataset_train.csv", encoding="UTF-8")
df_test = pandas.read_csv(data_dir_path + "dataset_test.csv", encoding="UTF-8")

# 'barajamos' el dataset de manera aleatoria para poder obtener luego
# un conjunto de desarrollo "dev" más representativo.
df_train.sample(frac = 1, random_state = 42)

#? df_train = df_train.sample(frac=1, random_state=42)

# Para poder entrenar es necesario codificar las etiquetas como números.
# Codificaremos los negativos con 0 y los positivos con 1 según lo definido
# en los diccionarios label2id e id2label
df_train['_label'] = df_train['label'].apply(lambda x: label2id[x])
df_test['_label'] = df_test['label'].apply(lambda x : label2id[x])

p_train = 0.80    # Porcentaje de train.
p_eval = 0.20     # Porcentaje de eval.

from sklearn.model_selection import train_test_split
df_train, df_eval = train_test_split (df_train, test_size = p_eval)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para evaluar: ", len(df_eval))
print("Ejemplos usados para test: ", len(df_test))

Ejemplos usados para entrenar:  3336
Ejemplos usados para evaluar:  835
Ejemplos usados para test:  1788


In [ ]:
!pip install evaluate


### 1.2.3 Cargamos modelo a ajustar y etiquetas.

In [ ]:
# Importamos las librerías necesarias
from transformers import AutoModelForSequenceClassification
import torch
import numpy as np
import json
import random
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from evaluate import load
from sklearn.metrics import classification_report

# Fijamos semilla para garantizar reproducibilidad.
# Lo importante no es el valor de # la semilla, sino el hecho de fijarla.
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


In [ ]:
# Definimos una clase para preparar los datasets y una función para calcular
# la evaluación de las métricas durante el entrenamiento.

# Preparamos los datasets a partir de la clase base de torch para modelar
# datasets (si hubiéramos usado TensorFlow hubiéramos usado la clase
# correspondiente en dicha librería).

class PLNEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def compute_metrics(eval_pred, metric_name = "accuracy"):
    '''
    Calcula la evaluación de las métricas durante el entrenamiento.

    Args:
        eval_pred: predicciones del modelo.
        metric_name: nombre de la métrica a calcular.

    Returns:
        diccionario con las métricas calculadas.
    '''

    # carga la métrica de la librería evaluation.
    metric = load(metric_name)

    # extrae de las predicciones del modelo las
    # probabilidades predichas y las etiquetas verdaderas
    logits, labels = eval_pred

    # convierte las proabilidades (logits) en predicciones)
    predictions = np.argmax(logits, axis=-1)

    # imprime informe de resultados de la clasificación.
    print(classification_report(labels, predictions, digits=6))

    return metric.compute(predictions=predictions, references=labels)


In [ ]:
# Definimos los parámetros para el entrenamiento.

training_args = transformers.TrainingArguments (
  # donde guardar el modelo entrenado
  output_dir = data_dir_path + './results',

  # donde guardar los logs del entrenamiento.
  logging_dir = data_dir_path + './logs',

  # número de épocas. Una época es un ciclo completo en el que el modelo ve
  # todos los datos de entrenamiento una vez
  num_train_epochs = 3,

  # Cuando evaluar y salvar el modelo (estrategia de evaluación)
  # (en cada época o en cada paso)
  eval_strategy = "epoch",
  save_strategy = "epoch",

  # Tamaños del batch para entrenamiento y evaluación.
  per_device_train_batch_size = 16,
  per_device_eval_batch_size = 64,

  # Métrica usada para elegir el mejor modelo durante el aprendizaje
  metric_for_best_model = "eval_accuracy",

  # Máximo número de checkpoints a salvar
  save_total_limit = 1,

  # Carga el mejor modelo al final
  load_best_model_at_end=True,

  # Desactiva wandb (Weights and Biases)
  report_to = "none"
)

In [ ]:
# PREPARAMOS LOS DATA SETS Y EL "ENTRENADOR"

# Tokenizamos los textos usados para el entrenamiento. Los ajustamos al tamaño
# del modelo truncando los que son demasiado largos y rellenando (padding) los
# demasiado cortos.
tokenized_train_dataset = tokenizer (df_train.tweet.tolist (),
                                     truncation=True, padding = True)
tokenized_eval_dataset = tokenizer (df_eval.tweet.tolist (),
                                    truncation=True, padding = True)
tokenized_test_dataset = tokenizer (df_test.tweet.tolist (),
                                    truncation=True, padding = True)

# Creamos los datasets para hacer el finetuning: entrenamiento, eval y test.
train_dataset = PLNEDataset (tokenized_train_dataset, df_train._label.tolist())
eval_dataset = PLNEDataset (tokenized_eval_dataset, df_eval._label.tolist())
test_dataset = PLNEDataset (tokenized_test_dataset, df_test._label.tolist())

# Usamos DataCollator para padding dinámico.
# Aseguramos así que todas las secuencias de un batch tienen la misma longitud,
# que es condición necesaria para el buen rendimiento de las GPUs.
# ?: parece redundante con la tokenización realizada al principio.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer (
    model = modelo_preentrenado,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
    data_collator=data_collator
)




In [ ]:
# ENTRENAMOS EL MODELO:
trainer.train()

print ("PREDICCIONES SOBRE EVAL")
modelo_preentrenado.eval()
print (json.dumps (trainer.evaluate (), indent = 2))

# Salvamos el modelo reentrenado
modelo ='modelo_reentrenado_torch'
trainer.save_model (modelo)
tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))



Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.374182,0.839521
2,No log,0.362553,0.883832
3,0.271500,0.453295,0.879042


              precision    recall  f1-score   support

           0   0.829016  0.613027  0.704846       261
           1   0.842679  0.942509  0.889803       574

    accuracy                       0.839521       835
   macro avg   0.835847  0.777768  0.797324       835
weighted avg   0.838408  0.839521  0.831990       835

              precision    recall  f1-score   support

           0   0.820312  0.804598  0.812379       261
           1   0.911917  0.919861  0.915872       574

    accuracy                       0.883832       835
   macro avg   0.866115  0.862229  0.864125       835
weighted avg   0.883284  0.883832  0.883522       835

              precision    recall  f1-score   support

           0   0.810078  0.800766  0.805395       261
           1   0.909879  0.914634  0.912250       574

    accuracy                       0.879042       835
   macro avg   0.859978  0.857700  0.858823       835
weighted avg   0.878683  0.879042  0.878850       835

PREDICCIONES SOBRE 

              precision    recall  f1-score   support

           0   0.820312  0.804598  0.812379       261
           1   0.911917  0.919861  0.915872       574

    accuracy                       0.883832       835
   macro avg   0.866115  0.862229  0.864125       835
weighted avg   0.883284  0.883832  0.883522       835

{
  "eval_loss": 0.3625525236129761,
  "eval_accuracy": 0.8838323353293414,
  "eval_runtime": 7.1346,
  "eval_samples_per_second": 117.036,
  "eval_steps_per_second": 1.962,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
              precision    recall  f1-score   support

           0   0.838583  0.759358  0.797007       561
           1   0.894531  0.933170  0.913442      1227

    accuracy                       0.878635      1788
   macro avg   0.866557  0.846264  0.855224      1788
weighted avg   0.876977  0.878635  0.876910      1788

{
  "test_loss": 0.3789728879928589,
  "test_accuracy": 0.8786353467561522,
  "test_runtime": 13.3713,
  "test_samples_per_second":

Volvemos a clasificar las frases de antes y vemos que ahora, con el entrenamiento, dan unos resultados mucho mejores.

In [ ]:
# Cargamos el modelo ya con el FineTuning hecho
#modelo ='modeloReentrenadoPytorch'
modelo = 'modelo_reentrenado_torch'

bert_class_model = AutoModelForSequenceClassification. \
                       from_pretrained(modelo, num_labels=NUM_LABELS)

# Cargamos el Tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelo)

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST
for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = bert_class_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= bert_class_model.config.id2label[predicted_class_id]
  print(text,'=>', predicted_class_id, '=>', prediction, "  ", logits.softmax(1))


hay muchos más muertos por covid => 0 => negative    tensor([[0.9393, 0.0607]])
el número de afectados por covid aumenta => 0 => negative    tensor([[0.9277, 0.0723]])
vamos a salir de la pandemia => 1 => positive    tensor([[0.0225, 0.9775]])
ánimo a todos => 1 => positive    tensor([[0.0119, 0.9881]])


## Apartado 1.3 Compatibilidad entre modelos Pytorch y Tensorflow

Se pueden cargar modelos de Pytorch en Tensorflow y viceversa. A continuación cargamos el modelo entrenado en Pytorch para evaluar las frases de los textos de antes y ver su clasificación.

In [ ]:
import tensorflow as tf

# Probar a inferir nuevas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]


#tf_model = TFAutoModelForSequenceClassification.from_pretrained('modeloReentrenadoPytorch')
tf_model = TFAutoModelForSequenceClassification.from_pretrained('modelo_reentrenado_torch')

# Imprimimos las predicciones obtenidas
for text in textos:
  predict_input = tokenizer.encode (text, truncation=True, padding=True, return_tensors="tf")
  tf_output = tf_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
  print(text,'=>', tf_prediction)

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


1/1 [==============================] - 8s 8s/step
hay muchos más muertos por covid => [0.93929744 0.06070262]
1/1 [==============================] - 3s 3s/step
el número de afectados por covid aumenta => [0.9276507  0.07234934]
1/1 [==============================] - 0s 41ms/step
vamos a salir de la pandemia => [0.0224824  0.97751755]
1/1 [==============================] - 0s 46ms/step
ánimo a todos => [0.01193382 0.9880662 ]


## Apartado 1.4 Probamos a hacer Finetunning de un modelo "destilado" de BERT

En el siguiente bloque de código hacemos el mismo entrenamiento en Pytorch pero con el modelo 'CenIA/distillbert-base-spanish-uncased' que es más pequeño y rápido de entrenar


In [ ]:
path_distilbert_model = 'CenIA/distillbert-base-spanish-uncased'

# Cargamos el modelo para clasificación en Pytorch
distilbert_class_model_pytorch = AutoModelForSequenceClassification. \
                   from_pretrained(path_distilbert_model, num_labels=NUM_LABELS)

distilbert_class_model_pytorch.config.id2label = id2label
distilbert_class_model_pytorch.config.label2id = label2id

# Cargamos el tokenizer de este modelo
distilbert_tokenizer = AutoTokenizer.from_pretrained(path_distilbert_model)

metric_name = "eval_accuracy"

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = data_dir_path + './results',
  logging_dir = data_dir_path + './logs',
  num_train_epochs = 3,
  eval_strategy = "epoch",
  save_strategy = "epoch",
  per_device_train_batch_size = 16,
  per_device_eval_batch_size = 64,
  metric_for_best_model = "eval_accuracy",
  save_total_limit = 1,
  load_best_model_at_end = True,
  report_to = "none"
)


tokenized_train_dataset = distilbert_tokenizer (df_train.tweet.tolist (),
                                                truncation=True, padding = True)
tokenized_eval_dataset = distilbert_tokenizer (df_eval.tweet.tolist (),
                                               truncation=True, padding = True)
tokenized_test_dataset = distilbert_tokenizer (df_test.tweet.tolist (),
                                               truncation=True, padding = True)

# Preparamos los 3 datasets para hacer el finetuning
train_dataset = PLNEDataset (tokenized_train_dataset, df_train._label.tolist())
eval_dataset = PLNEDataset (tokenized_eval_dataset, df_eval._label.tolist())
test_dataset = PLNEDataset (tokenized_test_dataset, df_test._label.tolist())

# Usamos DataCollator para padding dinámico
data_collator = DataCollatorWithPadding(tokenizer=distilbert_tokenizer)

trainer = Trainer (
    model = distilbert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
    data_collator = data_collator
)



config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at CenIA/distillbert-base-spanish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
trainer.train()

print ("PREDICCIONES SOBRE EVAL")
distilbert_class_model_pytorch.eval()
print (json.dumps (trainer.evaluate (), indent = 2))

# Salvamos el modelo reentrenado
modelo ='modelo_reentrenado_torch_distilbert_sp'
trainer.save_model (modelo)
distilbert_tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.321892,0.856287
2,No log,0.388807,0.877844
3,0.255600,0.513589,0.877844


              precision    recall  f1-score   support

           0   0.834171  0.656126  0.734513       253
           1   0.863208  0.943299  0.901478       582

    accuracy                       0.856287       835
   macro avg   0.848689  0.799713  0.817996       835
weighted avg   0.854410  0.856287  0.850889       835

              precision    recall  f1-score   support

           0   0.810700  0.778656  0.794355       253
           1   0.905405  0.920962  0.913118       582

    accuracy                       0.877844       835
   macro avg   0.858052  0.849809  0.853736       835
weighted avg   0.876710  0.877844  0.877133       835

              precision    recall  f1-score   support

           0   0.818565  0.766798  0.791837       253
           1   0.901338  0.926117  0.913559       582

    accuracy                       0.877844       835
   macro avg   0.859952  0.846458  0.852698       835
weighted avg   0.876258  0.877844  0.876678       835

PREDICCIONES SOBRE 

              precision    recall  f1-score   support

           0   0.810700  0.778656  0.794355       253
           1   0.905405  0.920962  0.913118       582

    accuracy                       0.877844       835
   macro avg   0.858052  0.849809  0.853736       835
weighted avg   0.876710  0.877844  0.877133       835

{
  "eval_loss": 0.3888072967529297,
  "eval_accuracy": 0.8778443113772455,
  "eval_runtime": 6.7746,
  "eval_samples_per_second": 123.254,
  "eval_steps_per_second": 2.067,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
              precision    recall  f1-score   support

           0   0.841141  0.736185  0.785171       561
           1   0.885891  0.936430  0.910460      1227

    accuracy                       0.873602      1788
   macro avg   0.863516  0.836308  0.847815      1788
weighted avg   0.871850  0.873602  0.871149      1788

{
  "test_loss": 0.38260194659233093,
  "test_accuracy": 0.8736017897091722,
  "test_runtime": 8.1682,
  "test_samples_per_second":

Por último probamos a inferir las nuevas frases con este modelo entrenado a partir de un modelo destilado que es bastante más pequeño.

In [ ]:
# Mover el modelo a la GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distilbert_class_model_pytorch.to(device)

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST

for text in textos:
  # Movemos los tensores del tokenizer a la GPU
  inputs = distilbert_tokenizer(text, return_tensors="pt").to("cuda")
  with torch.no_grad():
    logits = distilbert_class_model_pytorch(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= distilbert_class_model_pytorch.config.id2label[predicted_class_id]
  print(text,'=>', predicted_class_id, '=>', prediction, "  ", logits.softmax(1))

hay muchos más muertos por covid => 1 => positive    tensor([[0.4963, 0.5037]], device='cuda:0')
el número de afectados por covid aumenta => 0 => negative    tensor([[0.5360, 0.4640]], device='cuda:0')
vamos a salir de la pandemia => 1 => positive    tensor([[0.4845, 0.5155]], device='cuda:0')
ánimo a todos => 1 => positive    tensor([[0.4622, 0.5378]], device='cuda:0')
